In [1]:
!pip install  --upgrade "transformers"   "datasets"  "accelerate"  "evaluate"  "bitsandbytes"  "trl"  "peft" 
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 779.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pyngrok

In [3]:
!ngrok config add-authtoken 2fit1joWazA2g1HFUROudShO7KI_2siPG2oNzQDpo8RdgjVCm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from flask import Flask, request, jsonify
from time import time
import torch
import transformers
from pyngrok import ngrok
import threading
import pandas as pd
import json
import numpy as np
import os
from huggingface_hub import notebook_login

from peft import LoraConfig, PeftConfig, PeftModel
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [5]:
app = Flask(__name__)
pipeline = None

In [6]:
temperature=0.7
max_length=256
top_p=0.9
model_name = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

In [7]:
# peft_model_id = "Shivam17818/llama3-8b-instruct-text-to-sql-order_values"
# config = PeftConfig.from_pretrained(peft_model_id)

In [8]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
# model.load_adapter(peft_model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          padding_side = "right",
                                          add_eos_token = True)
tokenizer.pad_token = tokenizer.eos_token

In [11]:
try:
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer, 
        torch_dtype=torch.float16,
        device_map="auto",
)
except Exception as e:
    print(f"Failed to load the model: {e}") 
    raise RuntimeError("Failed to initialize the model. Please check the model path and configurations.")


2024-09-03 08:30:55.583166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 08:30:55.583328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 08:30:55.711727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:

def query_model(input_prompt,system_message):
    start_time = time()
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": input_prompt},
    ]

    try:
        prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
    except Exception as e:
        raise RuntimeError(f"Error applying chat template: {e}")
    
    terminators = [
        pipeline.tokenizer.eos_token_id,

        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    try:
        sequences = pipeline(
            prompt,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
            eos_token_id=terminators,
            max_new_tokens=max_length,
            return_full_text=False,
            pad_token_id=pipeline.model.config.eos_token_id
    )
    except Exception as e:
        raise RuntimeError(f"Error during model generation: {e}")
    
    generated_query = sequences[0]['generated_text']
    end_time = time()
    ttime = round(end_time - start_time, 2)

    return generated_query, ttime

In [13]:
@app.route('/v1/llama/response', methods=['POST'])
def query():
    if pipeline is None:
        return jsonify({"error": "Model not initialized properly"}), 500

    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "Invalid JSON format"}), 400
        
        input_prompt = data.get('input_prompt')
        system_message = data.get('system_message')
        

        if not input_prompt or not system_message:
            return jsonify({"error": "Missing 'input_prompt' and 'system_message'"}), 400
        
        generated_query, ttime = query_model(input_prompt,system_message)
        print(generated_query)

        return jsonify({
            "generated_query":generated_query,
            "total_time": f"{ttime} sec."
        })
    
    except KeyError as e:
        return jsonify({"error": f"Missing required parameter: {str(e)}"}), 400
    except RuntimeError as e:
        return jsonify({"error": str(e)}), 500
    except Exception as e:
        return jsonify({"error": f"An unexpected error occurred: {str(e)}"}), 500

@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Not Found"}), 404

@app.errorhandler(500)
def internal_error(error):
    return jsonify({"error": "Internal Server Error"}), 500

def run_flask_app():
    app.run(host='0.0.0.0', port=3008)

if __name__ == '__main__':
    ngrok_tunnel = ngrok.connect(3008)
    print(f'Public URL: {ngrok_tunnel.public_url}')
    thread = threading.Thread(target=run_flask_app)
    thread.start()


Public URL: https://09f9-34-121-197-251.ngrok-free.app
